In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# URL yang akan di scraping
URL = "https://liquipedia.net/mobilelegends/Portal:Teams"

# Header untuk menghindari blocking 403
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Linux; Cashtree) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.114 Mobile Safari/537.36"
}

In [3]:
response = requests.get(URL, headers=HEADERS, timeout=15)

In [4]:
if response.status_code != 200:
    print(f"Error: HTTP {response.status_code}")

soup = BeautifulSoup(response.content, "html.parser")

In [5]:
teams_content = soup.find("div", {"id": "mw-content-text"})

In [13]:
teams_header = teams_content.find("h2")

In [15]:
teams_header

<h2><span class="mw-headline" id="Top_10_Most_Earnings">Top 10 Most Earnings</span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/mobilelegends/index.php?title=Portal:Teams&amp;action=edit&amp;section=1" title="Edit section: Top 10 Most Earnings">edit</a><span class="mw-editsection-bracket">]</span></span></h2>

In [16]:
team_list = teams_header.find_next_sibling()

In [18]:
team_list_content = team_list.find_all("div", {"class": "team-portal-list"})

In [33]:
base_img_url = "https://liquipedia.net"

## Top 10 Most Earning

In [50]:
top_10_most_earning_team = []

In [51]:
for team in team_list_content:
    team_earning = team.find_all("span")
    rank = team.find("span", {"class": "tp-rank-number"}).text
    name = team.find("span", {"class": "tp-team-name"}).text
    img = f"{base_img_url}{team.select_one("span.team-template-team-icon span.team-template-darkmode img")["srcset"]}".split("1.5x,")[0].strip()
    total_earn = team.find("span", {"class": "tp-total-earnings"}).text.replace("Total Earnings:", "").strip()
    earning = {"rank": rank, "name": name, "img": img, "total_earnings": total_earn}
    top_10_most_earning_team.append(earning)  

In [ ]:
top_10_most_earning_team

## All Region Team

In [53]:
team_navigation = teams_content.find("div", {"class": "tabs-static"})

In [55]:
list_nav = team_navigation.find_all("li")

In [58]:
skip_nav = ["Teams"]

In [172]:
team_region = []

In [ ]:
team_region[-1]

In [71]:
region_url = f"{base_img_url}{list_nav[1].find("a")["href"]}"

In [ ]:
for ls in list_nav:
    
    if ls.text not in skip_nav:
        region_url = f"{base_img_url}{ls.find("a")["href"]}"
        regional_name = ls.find("a").text
        team_per_regional = get_team_by_region(region_url, regional_name)
        team_region.append(team_per_regional)

In [61]:
team_by_country_content = teams_content.find("div", {"class": "mw-content-ltr"})

In [63]:
team_by_country_content.find_all("h3")

[]

In [159]:
region_url = "https://liquipedia.net/mobilelegends/Portal:Teams/China"

In [171]:
def get_team_by_region(url, region):
    print("=" *20)
    print(f"Regional : {region}")
    data = {"region": region, "countries": []}
    is_one_region = False
    team_request = requests.get(url, headers=HEADERS, timeout=15)
    if team_request.status_code != 200:
        print(f"Error: HTTP {team_request.status_code}")

    soup = BeautifulSoup(team_request.content, "html.parser")
    regional_team = soup.find("div", {"class": "mw-content-ltr"})
    list_team_by_regional = regional_team.find_all("h3")
    if len(list_team_by_regional) == 0:
        list_team_by_regional = regional_team.find_all("h2")
        is_one_region = True

    
    for team in list_team_by_regional:
        region = {"name": team.text.replace("[edit]", "") if not is_one_region else region, "teams": []}
        print(f"Country : {team.text}")
        team_player_per_country_content = team.find_next_sibling().find_all("div", {"class": "template-box"})
        for region_team in team_player_per_country_content:
            team_data = {}
            # get team name and team url
            team_name = region_team.select_one("table.wikitable tbody tr th span.team-template-team-standard span.team-template-text a")
            if team_name is not None:
                team_url = f"{base_img_url}{team_name["href"] if team_name else "Unknown"}"
                name = team_name.text
                team_data["name"] = name
                team_data["url"] = team_url
    
            # get team player and player url
            players = []
            players_content = region_team.find_all("tr")
            for player in players_content:
                player_detail_content = player.find("td")
                if player_detail_content is not None:
                    country = player_detail_content.select_one("span.flag")
                    if country is not None:
                        player_id = country.find_next_sibling()
                        player_detail = {"id": player_id.find("a").text,
                                     "player_url": f"{base_img_url}{player_id.find("a")["href"]}",
                                     "real_name": player_detail_content.find_next_sibling().text,
                                     "from": country.find("a")["title"],
                                     "flag": f"{base_img_url}{country.find("img")["src"]}" 
                                    }
                    players.append(player_detail)
            team_data["players"] = players
            region["teams"].append(team_data)
        data["countries"].append(region)
    print("=" *20)
    return data
    

In [160]:
china_region = requests.get(region_url, headers=HEADERS, timeout=15)
if china_region.status_code != 200:
    print(f"Error: HTTP {china_region.status_code}")
    
soup = BeautifulSoup(china_region.content, "html.parser")
regional_team_china = soup.find("div", {"class": "mw-content-ltr"})
list_team_by_regional = regional_team_china.find_all("h3")
if len(list_team_by_regional) == 0 :
    list_team_by_regional = regional_team_china.find_all("h2")
# print(len(list_team_by_regional))

In [ ]:
for team in list_team_by_regional:
    region = {"country": team.text.replace("[edit]", ""), "teams": []}
    print("=" *20)
    print(f"Country : {team.text}")
    team_player_per_country_content = team.find_next_sibling().find_all("div", {"class": "template-box"})
    for region_team in team_player_per_country_content:
        team_data = {}
        # get team name and team url
        team_name = region_team.select_one("table.wikitable tbody tr th span.team-template-team-standard span.team-template-text a")
        if team_name is not None:
            team_url = f"{base_img_url}{team_name["href"] if team_name else "Unknown"}"
            name = team_name.text
            team_data["name"] = name
            team_data["url"] = team_url
            # print(team_data)

        # get team player and player url
        players = []
        players_content = region_team.find_all("tr")
        for player in players_content:
            player_detail_content = player.find("td")
            if player_detail_content is not None:
                country = player_detail_content.select_one("span.flag")
                player_id = country.find_next_sibling()
                player_detail = {"id": player_id.find("a").text,
                                 "player_url": f"{base_img_url}{player_id.find("a")["href"]}",
                                 "real_name": player_detail_content.find_next_sibling().text,
                                 "from": country.find("a")["title"],
                                 "flag": f"{base_img_url}{country.find("img")["src"]}" 
                                }
                players.append(player_detail)
        team_data["players"] = players
        region["teams"].append(team_data)
    print(region["teams"])
    print("=" *20)
    